In [17]:
import pandas as pd
import numpy as np
import pickle

In [22]:
TRADES_DIR = '../TRADES/USD_TOM_trades_Micex_02_10.feather'
LOB_DIR = '../Data_lob/'

GOOD_LOBS_OUTPUT = '../0_CommonFiles/goodlobs.txt'
RANDOM_MOMENTS_ALL_OUTPUT = '../0_CommonFiles/random_moments_all.pkl'
RANDOM_MOMENTS_ALIGNED_OUTPUT = '../0_CommonFiles/random_moments_aligned_to_1000_2000.pkl'

TRADE_SIZE_MIN_INCLUSIVE = 1000
TRADE_SIZE_MAX_INCLUSIVE = 2000

# Part 1. Select big trades

In [4]:
trades = pd.read_feather(TRADES_DIR)

In [5]:
trade_temp = trades.groupby(['Time', 'AGGRESSOR_SIDE'], as_index=False).sum()

In [6]:
volume_weighted_trade_price = trade_temp['TRADE_VALUE'] / trade_temp['SIZE'] / 1000.0

In [7]:
trade_temp['PRICE'] = volume_weighted_trade_price
# round to 0.01 of cent
trades['PRICE'] = trades['PRICE'].round(4)

In [8]:
trades = trade_temp

In [9]:
del trade_temp

In [11]:
#look only on big trades
trades = trades[(trades.SIZE >= TRADE_SIZE_MIN_INCLUSIVE) & (trades.SIZE <= TRADE_SIZE_MAX_INCLUSIVE)]

In [15]:
with open(RANDOM_MOMENTS_ALL_OUTPUT, 'rb') as file:
    moments = pickle.load(file)

In [18]:
look_back_time = np.timedelta64(300, 's')

# Part 2. Align random moments to big trades Timestamp

In [26]:
moments2 = []
for moment in moments:
    index1 = trades.Time.searchsorted(moment - look_back_time, side='left')
    index2 = trades.Time.searchsorted(moment, side='right')

    
    if index2 - index1 > 0:
        trades_before = trades.iloc[index1 : index2]
        # get the big trade, that nearest to 'moment'
        moments2.append(trades_before.Time.iloc[-1])
    else:
        trades_before = trades.iloc[index1 : index2]
        if len(trades_before) > 0:
            print(trades_before)
            raise ValueError('error')
            
with open(RANDOM_MOMENTS_ALIGNED_OUTPUT, 'wb') as file:
    pickle.dump(moments2, file)

In [27]:
len(moments2)

2712

In [25]:
len(moments)

5280